In [ ]:
import sys; sys.version_info

In [ ]:
import os
import pandas as pd
import torch
from torchvision import transforms
from tqdm.notebook import tqdm
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
%run ../covid_uc.py

### Calculate mean and std

In [ ]:
%run ../../utils/images.py

In [ ]:
labels_fpath = os.path.join(DATASET_DIR, 'metadata.csv')
metadata = pd.read_csv(labels_fpath, index_col=0)
metadata.head()

In [ ]:
def get_image_names(frontal_only=False):
    if frontal_only:
        df = metadata.loc[metadata['view'].str.contains('P')]
    else:
        df = metadata
    return list(df['image_name'])

In [ ]:
image_names_frontal = get_image_names(True)
image_names_all = get_image_names(False)
len(image_names_frontal), len(image_names_all)

In [ ]:
images_dir = os.path.join(DATASET_DIR, 'images')

In [ ]:
mean, std = compute_mean_std(ImageFolderIterator(images_dir, image_names_frontal), show=True)
mean, std

In [ ]:
mean, std = compute_mean_std(ImageFolderIterator(images_dir, image_names_all), show=True)
mean, std

### Test `CovidUCDataset` class

In [ ]:
%run ../covid_uc.py

In [ ]:
dataset = CovidUCDataset('test', image_size=(256, 256), frontal_only=True)
len(dataset)

In [ ]:
dataset._metadata_df

In [ ]:
for image, label in dataset:
    break

In [ ]:
label

In [ ]:
image.size()

In [ ]:
dataset.labels

In [ ]:
dataset.get_labels_presence_for('Non-COVID')

### Split

In [ ]:
%run ../covid_uc.py

In [ ]:
import random
import os
from collections import Counter

In [ ]:
LABEL_COL = 'Resultado consenso BSTI'

In [ ]:
def save_list(items, name):
    filepath = os.path.join(DATASET_DIR, f'{name}.txt')
    with open(filepath, 'w') as f:
        for item in items:
            f.write(f'{item}\n')

    print(f'List saved to: {filepath}')

In [ ]:
labels_fpath = os.path.join(DATASET_DIR, 'metadata.csv')
metadata = pd.read_csv(labels_fpath, index_col=0)
metadata.head()

In [ ]:
images_by_patient = metadata.groupby('ID')['image_name'].apply(list)
images_by_patient

In [ ]:
patients = list(set(metadata['ID']))
len(patients)

In [ ]:
VAL_SPLIT = 0
TEST_SPLIT = 0.1

In [ ]:
n_val = int(VAL_SPLIT * len(patients))
n_test = int(TEST_SPLIT * len(patients))
n_val, n_test

In [ ]:
val_test_patients = random.sample(patients, n_val + n_test)
len(val_test_patients)

In [ ]:
train_patients = [pat for pat in patients if pat not in val_test_patients]
len(train_patients)

In [ ]:
combine_images = lambda pats: sum((images_by_patient[pat] for pat in pats), [])
count_images = lambda pats: sum(len(images_by_patient[pat]) for pat in pats)

In [ ]:
count_images(train_patients), count_images(val_test_patients)

In [ ]:
train_images = combine_images(train_patients)
val_test_images = combine_images(val_test_patients)

In [ ]:
filter_meta = lambda images: metadata.loc[metadata['image_name'].isin(images)]

In [ ]:
train_df = filter_meta(train_images)
val_test_df = filter_meta(val_test_images)
# train_df

In [ ]:
Counter(train_df[LABEL_COL])

In [ ]:
total = (94 + 678 + 23)
mult = 100/total
94 * mult, 678 * mult, 23 * mult

In [ ]:
Counter(val_test_df[LABEL_COL])

In [ ]:
4 / 88 * 100, 79 / 88 * 100, 5 / 88 * 100

In [ ]:
len(train_images), len(val_test_images)

In [ ]:
val_images = val_test_images[:n_val]
test_images = val_test_images[n_val:]
len(val_images), len(test_images)

In [ ]:
save_list(train_images, 'train')
save_list(val_images, 'val')
save_list(test_images, 'test')